# 📦 Setup

In [3]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [ ]:
PATH = './'
TRAIN = '../input/train_v2'
TEST = '../input/test_v2/'
SEGMENTATION = '../input/train_ship_segmentations_v2.csv'

exclude_list = ['6384c3e78.jpg','13703f040.jpg', '14715c06d.jpg',  '33e0ff2d5.jpg',
                '4d4e09f2a.jpg', '877691df8.jpg', '8b909bb20.jpg', 'a8d99130e.jpg', 
                'ad55c3143.jpg', 'c8260c541.jpg', 'd6c7f17c7.jpg', 'dc3e7c901.jpg',
                'e44dffe88.jpg', 'ef87bad36.jpg', 'f083256d8.jpg']

In [ ]:
n_workers = 4 
architecture = resnet34 

In [ ]:
# Get all image names from train and test dataset 
train_names = [f for f in os.listdir(TRAIN)]
test_names = [f for f in os.listdir(TEST)]

# exclued corrupted images  
for el in exclude_list:
    if(el in train_names): train_names.remove(el)
    if(el in test_names): test_names.remove(el)

# split train dataset
tr_n, val_n = train_test_split(train_names, test_size=0.05, random_state=42)

# load segmentation dataset 
segmentation_df = pd.read_csv(os.path.join(PATH, SEGMENTATION)).set_index('ImageId')

In [ ]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz))
    
    def get_y(self, i):
        if(self.path == TEST): return 0
        masks = self.segmentation_df.loc[self.fnames[i]]['EncodedPixels']
        if(type(masks) == float): return 0 # NAN - no ship 
        else: return 1
    
    def get_c(self): return 2 #number of classes, ship / no ship

In [ ]:
def get_data(sz,bs):
    """
        [sz] - image size
        [bs] - batch size  
    """
    # data augmentation
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    
    tfms = tfms_from_model(architecture, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    
    md = ImageClassifierData(PATH, ds, bs, num_workers=n_workers, classes=None)
    return md

# 🎯 Prediction

In [ ]:
image_size = 256 
batch_size = 64 

md = get_data(image_size,batch_size)
learn = ConvLearner.pretrained(architecture, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam

In [ ]:
learn.load('Resnet34_lable_384_1')

In [ ]:
log_preds,y = learn.predict_with_targs(is_test=True)
probs = np.exp(log_preds)[:,1]
pred = (probs > 0.5).astype(int)

In [ ]:
df = pd.DataFrame({'id':test_names, 'p_ship':probs})
df.to_csv('ship_detection.csv', header=True, index=False)

In [ ]:
df.head(40)